In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import csv 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
import numpy as np
import h5py
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
!pip install xcs

In [ ]:
import xcs
from xcs import XCSAlgorithm
from xcs.scenarios import MUXProblem, ScenarioObserver
import logging
logging.root.setLevel(logging.INFO)
import random
from xcs.scenarios import Scenario
from xcs.bitstrings import BitString

In [ ]:
import pandas as pd
df = pd.read_csv("../input/bankmarketing/bank marketing.csv")
df.head(5)


In [ ]:
df.drop(['job', 'marital','education','contact','month','poutcome'], axis = 1, inplace=True) 
df.head(5)

In [ ]:
print(df["duration"].min())

In [ ]:
df["age"] = pd.cut(df["age"],4,labels=[0,1,2,3])
#df.head(5)
df_new["default"] = pd.cut(df["default"],3,labels=[0,1,2,])
df_new["balance"] = pd.cut(df["balance"],5,labels=[0,1,2,3,4])
df_new["housing"] = pd.cut(df["housing"],4,labels=[0,1,2,3])
df_new["loan"] = pd.cut(df["loan"],4,labels=[0,1,2,3])
df_new["day"] = pd.cut(df["day"],4,labels=[0,1,2,3])
df_new["duration"] = pd.cut(df["duration"],4,labels=[0,1,2,3])
df_new["campaign"] = pd.cut(df["campaign"],3,labels=[0,1,2])
df_new["pdays"] = pd.cut(df["pdays"],3,labels=[0,1,2])
df_new["previous"] = pd.cut(df["previous"],3,labels=[0,1,2])
df_new["market?"] = pd.cut(df["market?"],3,labels=[0,1,2])


In [ ]:
df_new.head(10)

In [ ]:
df1 = pd.get_dummies(df_new["age"], prefix = 'age')
df2 = pd.get_dummies(df_new["default"], prefix = 'default')
df3 = pd.get_dummies(df_new["balance"], prefix = 'balance')
df4 = pd.get_dummies(df_new["housing"], prefix = 'housing')
df5 = pd.get_dummies(df_new["loan"], prefix = 'loan')
df6 = pd.get_dummies(df_new["day"], prefix = 'day')
df7 = pd.get_dummies(df_new["duration"], prefix = 'duration')
df8 = pd.get_dummies(df_new["campaign"], prefix = 'campaign')
df9 = pd.get_dummies(df_new["pdays"], prefix = 'pdays')
df10 = pd.get_dummies(df_new["previous"], prefix = 'previous')
df11 = pd.get_dummies(df_new["market?"], prefix = 'market?')

In [ ]:
BankDataFrame = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11], axis=1)
BankDataFrame.head(10)

In [ ]:
print(BankDataFrame.shape)

In [ ]:
pid_dataset = BankDataFrame.values
arr = []
for i in range(len(pid_dataset)):
    arr.append(''.join([str(x) for x in pid_dataset[i]]))

In [ ]:
class BankDataset(Scenario):
    def __init__(self, training_cycles=1000, input_size=200, inputs=[]):
        self.input_size = input_size
        self.possible_actions = (1, 0)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle_index = random.randrange(input_size)
        self.needle_value = None
        self.inputs = inputs
    
    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        self.needle_index = random.randrange(self.input_size)
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        random.seed()
        i = random.randint(0,len(self.inputs) - 1)
        c_feat = BitString(self.inputs[i][:-1])
        self.needle_value = self.inputs[i][-1]
        return c_feat
        
    def execute(self, action):
        self.remaining_cycles -= 1
        return int(action) == int(self.needle_value)

In [ ]:
inp = len(arr[0])-1
problem = BankDataset(training_cycles=10000, input_size=inp, inputs = arr)

algorithm = xcs.XCSAlgorithm()

# Default parameter settings in test()
algorithm.exploration_probability = .1
algorithm.discount_factor = 0
algorithm.do_ga_subsumption = True
algorithm.do_action_set_subsumption = True

# Modified parameter settings
algorithm.ga_threshold = 1
algorithm.crossover_probability = .5
algorithm.wildcard_probability = .5
algorithm.deletion_threshold = 2 
algorithm.subsumption_threshold = 10         # theta_sub
algorithm.mutation_probability = .03

In [ ]:
model = algorithm.new_model(ScenarioObserver(problem))
history = model.run(ScenarioObserver(problem), learn=True)

In [ ]:
print(model)

In [ ]:
print(len(model))

In [ ]:
for rule in model:
    if rule.fitness > .5 and rule.experience >= 10:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)  